# Initialize main functions

In [ ]:
%reset -f
#usage: python main.py
#output: solution to the Poission-Boltzmann equation
#!need to verify u[:] when copy variables especially in functions
#!nonzero beta and gamma was not implemented yet

import numpy as np
import numpy.matlib
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.linalg import block_diag
import time

#Domain discretization and required variables (Jin 2002 P167)
#The volume V is subdivided into a number tetrahedral elements and
#the surface is broken into a number of triangular elements
#(1) array n(i,e) of size 4xM
#The element numbers and node numbers can be related by a 4xM integer array n(i,e)
#where i=1,2,3,4, e=1,2,3,...,M, and M denotes the total number of volume elements
#The array n(i,e) stores the global node number of the i^th node of the e^th element
#(2) vector nd(i) of length N1
#To facilitate imposition of the Dirichlet boundary condition on surface S1
#the integer vector nd(i) stores the global node number of the nodes that reside on
#the surface S1, where i=1,2,3,...,N1, and N1 denotes the total number of such nodes
#(3) array ns(i,e) of size 3xMs - not defined yet
#For the treatment of the boundary condition of the third kind on surface S2
#the 3xMs integer array ns(i,e) holds information related to the surface triangular
#elements and their associated nodes residing on the surface S2, where i=1,2,3,
#e=1,2,3,...,Ms,and Ms is the total number of surface triangular elements on S2
#The array ns(i,e) stores the global number of the i^th node of the s^th elements
#(4) Other data that are needed include the coordinates of each node, the value of
#PDE coefficients for each volume elements, the prescribed value of u for each node
#on S1, and the coefficients of the boundary condition of the third kind
#for each surface triangular elements on S2

#Useful links
#(1) Quadrature on tetrahedra
#https://www.cfd-online.com/Wiki/Code:_Quadrature_on_Tetrahedra
#(2) Quadpy
#https://github.com/nschloe/quadpy
#(3) Shape function
#https://www.iue.tuwien.ac.at/phd/orio/node48.html
#(4) Another document about reference tetrahedra
#https://people.sc.fsu.edu/~jburkardt/presentations/cg_lab_fem_basis_tetrahedron.pdf
#(5) COMSOL for static nonlinear finite element problems
#https://www.comsol.com/blogs/solving-nonlinear-static-finite-element-problems/
#(6) Set values in Python sparse matrix
#http://seanlaw.github.io/2019/02/27/set-values-in-sparse-matrix/
#(7) Molarity unit conversion
#http://web.pdx.edu/~justc/courses/RecombDNALab/MolBioWorksheet.pdf
#(8) Why stiffness matrix is non-symmetric
#https://www.quora.com/Why-is-the-stiffness-matrix-in-the-finite-element-method-symmetric
#(9) Contour plot of irregularly spaced data
#https://matplotlib.org/stable/gallery/images_contours_and_fields/irregulardatagrid.html
#(10) Three-Dimensional Plotting in Matplotlib
#https://jakevdp.github.io/PythonDataScienceHandbook/04.12-three-dimensional-plotting.html

#set physical constants and parameters
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kB=1.381e-23 #Boltzmann's constant [J/K]
kA=6.022e23 #Avogadro constant [1/mol]

def set_globvars(infile):
    global nodes,elements,faces,midpoints #mesh variables
    global node_flags,zones,face_flags #variables for performing tests
    global mask_e,mask_d,mask_s #boolean element/node/face indicies for volume/S1/S2
    global cx,cy,cz #PDE coefficients for scalar field
    global alphax,alphay,alphaz #PDE coefficients for scalar field
    global betax,betay,betaz #PDE coefficients for scalar field
    global gammax,gammay,gammaz #PDE coefficients for scalar field
    global a,f,g,q,s #PDE coefficients for scalar field
    global cx33,cy33,cz33 #PDE coefficients for vector field
    global alphax33,alphay33,alphaz33 #PDE coefficients for vector field
    global betax33,betay33,betaz33 #PDE coefficients for vector field
    global gammax33,gammay33,gammaz33 #PDE coefficients for vector field
    global a33,f3,g3,q33,s3 #PDE coefficients for vector field
    global rel_perm,tK,cinf,zval #physical properties
    global kd #scaling coefficients for distance
    
    #load pde coefficients
    print('Loading %s'%infile)
    print('')
    data=np.load(infile)
    nodes=data['nodes']
    elements=data['elements']
    faces=data['faces']
    
    node_flags=data['node_flags']
    zones=data['zones']
    face_flags=data['face_flags']
    
    mask_e=data['mask_e'] #computational element indices
    mask_d=data['mask_d'] #node indicies on Dirichlet boundary (S1)
    mask_s=data['mask_s'] #surface element indicies on the boundary of the third kind (S2)
    
    cx=data['cx']
    cy=data['cy']
    cz=data['cz']
    alphax=data['alphax']
    alphay=data['alphay']
    alphaz=data['alphaz']
    betax=data['betax']
    betay=data['betay']
    betaz=data['betaz']
    gammax=data['gammax']
    gammay=data['gammay']
    gammaz=data['gammaz']
    a=data['a']
    f=data['f']
    g=data['g']
    q=data['q']
    s=data['s']
    
    cx33=data['cx33']
    cy33=data['cy33']
    cz33=data['cz33']
    alphax33=data['alphax33']
    alphay33=data['alphay33']
    alphaz33=data['alphaz33']
    betax33=data['betax33']
    betay33=data['betay33']
    betaz33=data['betaz33']
    gammax33=data['gammax33']
    gammay33=data['gammay33']
    gammaz33=data['gammaz33']
    a33=data['a33']
    f3=data['f3']
    g3=data['g3']
    q33=data['q33']
    s3=data['s3']
    
    rel_perm=data['rel_perm']
    tK=data['tK']
    cinf=data['cinf']
    zval=data['zval']
    
    #compute middle point of each elements (efficiency to be improved)
    nelem=len(elements[:,0])
    midpoints=np.zeros((nelem,3))
    for i in range(0,nelem):
        for j in range(0,3):
            midpoints[i,j]=.25*sum(nodes[elements[i,:],j])
    
    nind_e=np.unique(elements[mask_e].flatten(order='C'))
    print('THE NUMBER OF COMPUTATIONAL NODES IS:',len(nind_e))
    print('THE NUMBER OF COMPUTATIONAL ELEMENTS IS:',np.count_nonzero(mask_e))
    print('THE NUMBER OF NODES ON S1 IS:',np.count_nonzero(mask_d))
    print('THE NUMBER OF ELEMENTS ON S2 IS:',np.count_nonzero(mask_s))
    print('')
    
    print('Scale the PDE equation for distance')
    debye_len=np.sqrt(rel_perm*perm0*kB*tK/2/zval**2/echarge**2/kA/cinf)
    kd=10**(-round(np.log10(debye_len))) #to be determined from debye length
    nodes=nodes*kd
    print('Distance scaling coefficient is: %.2e'%kd)
    print('')
    
    alphax=alphax/kd
    alphay=alphay/kd
    alphaz=alphaz/kd
    
    betax=betax/kd
    betay=betay/kd
    betaz=betaz/kd
    
    gammax=gammax/kd
    gammay=gammay/kd
    gammaz=gammaz/kd
    
    #make sure a,f,g,q were scaled by perm0 before
    #actually all PDE coefficients should be scaled by perm0
    #scale everything in main.py in later modifications
    #by adding an option scale_perm0 or not in build_pde.py
    #then call scale_pde function in main.py after set_globvars
    #unscale perm0 for now
    a=a/kd/kd
    f=f/kd/kd
    
    g=g/kd
    q=q/kd #not really tested this q
    
    #scale PDE coefficients for vector unknowns
    alphax33=alphax33/kd
    alphay33=alphay33/kd
    alphaz33=alphaz33/kd
    
    betax33=betax33/kd
    betay33=betay33/kd
    betaz33=betaz33/kd
    
    gammax33=gammax33/kd
    gammay33=gammay33/kd
    gammaz33=gammaz33/kd
    
    a33=a33/kd/kd
    f3=f3/kd/kd
    g3=g3/kd
    q33=q33/kd
    
    return

def solve_u(K,b):
    print('Calling sparse linear system solver')
    start=time.time()
    K.eliminate_zeros()
    u=spsolve(K,b)
    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')

    return u

def output_u(nodes,u,srvfile):
    output=np.genfromtxt(srvfile,skip_header=1)
    rec=output[:,1:4]*kd
    for i in range(len(rec)):
        dist2=(rec[i,0]-nodes[:,0])**2+(rec[i,1]-nodes[:,1])**2+(rec[i,2]-nodes[:,2])**2
        ind=np.argmin(dist2)
        output[i,5]=u[ind]

    outfile=srvfile+'.out'
    print('Saving results to %s'%outfile)
    np.savetxt(outfile,output,header=str(len(output)),comments='',
               fmt='%d\t%10.2e\t%10.2e\t%10.2e\t%10.2e\t%10.2e')
    print('')
    return output

#output a is complex if input u is complex
def build_a(u):
    #input u on nodes output a on nodes
    #a_nodes=2*echarge**2*cinf/kB/tK*np.cosh(echarge*u/kB/tK)
    
    #Given tK=293, un=39.591*zval*u
    #Given tK=293, zval=1, u=1, np.cosh(un)=7.822e16
    #Given tK=293, a=7.639e6*zval*cinf*np.cosh(un)
    #Given tK=293, zval=1, u=1, cinf=1, a=5.975e23
    ze=zval*echarge #ion valence times elementary charge
    un=ze*u/kB/tK #normalized potential
    #a=2*ze**2*kA*cinf/kB/tK*np.cosh(un)/perm0/kd/kd #scaled by perm0
    a=2*ze**2*kA*cinf/kB/tK*np.cosh(un)/kd/kd
        
    return a

#output f is complex if input u is complex
def build_f(u):
    #input u on nodes output f on nodes
    #f_nodes=-2*echarge*cinf*(np.sinh(echarge*u/kB/tK)-np.cosh(echarge*u/kB/tK)*echarge*u/kB/tK)
    
    #Given tK=293, un=39.591*zval*u
    #Given tK=293, zval=1, u=1, np.cosh(un)=7.822e16
    #Given tK=293, zval=1, u=1, np.sinh(un)=7.822e16
    #Given tK=293, f=-1.930e5*zval*cinf*(np.sinh(un)-np.cosh(un)*un)
    #Given tK=293, zval=1, u=1, cinf=1, f=-5.824e23
    ze=zval*echarge #ion valence times elementary charge
    un=ze*u/kB/tK #normalized potential
    #f=-2*ze*kA*cinf*(np.sinh(un)-np.cosh(un)*un)/perm0/kd/kd #scaled by perm0
    f=-2*ze*kA*cinf*(np.sinh(un)-np.cosh(un)*un)/kd/kd
    
    return f

#compute gradient of scalar field u
#output du is complex if input u is complex
def gradient(xn,yn,zn,u):
    #input u on nodes
    #output gradient in elements
    du=np.zeros(3) #du is constant inside an element
    Je=np.zeros((4,4))
    
    Je[0,:]=1
    Je[1,:]=xn #x-coordinates of four nodes in an element
    Je[2,:]=yn
    Je[3,:]=zn
    invJe=np.linalg.inv(Je)
    
    du[0]=np.sum(u[:]*invJe[:,1])
    du[1]=np.sum(u[:]*invJe[:,2])
    du[2]=np.sum(u[:]*invJe[:,3])
    
    du=du*kd #do not forget distance scaling
    return du

#convert node based potential to element based potential
#output ue is complex if input un is complex
def unode2uelem(xn,yn,zn,un):
    #input u on nodes
    #output u in elements
    Je=np.zeros((4,4))
    
    Je[0,:]=1
    Je[1,:]=xn #x-coordinates of four nodes in an element
    Je[2,:]=yn
    Je[3,:]=zn
    invJe=np.linalg.inv(Je)

    xr=.25*sum(xn)
    yr=.25*sum(yn)
    zr=.25*sum(zn)
    ue=sum((invJe[:,0]+invJe[:,1]*xr+invJe[:,2]*yr+invJe[:,3]*zr)*un)
    return ue

#evaluate basis functions at (xr,yr,zr) given tetrahedra nodes (xn,yn,zn) (not tested or used so far)
def basis(xn,yn,zn,xr,yr,zr):
    Je=np.zeros((4,4))
    
    Je[0,:]=1
    Je[1,:]=xn #x-coordinates of four nodes in an element
    Je[2,:]=yn
    Je[3,:]=zn
    invJe=np.linalg.inv(Je)
    Ne=invJe.dot([1,xr,yr,zr])
    
    return Ne

#build system of equations for vector field (now testing)
def build_system3(nodes,elements,faces,cx33,cy33,cz33,a33,f3,g3,q33,s3):    
    nind_e=np.unique(elements[mask_e].flatten(order='C'))
    nind_map_global_to_local=-np.ones(len(nodes),dtype=int)
    nind_map_global_to_local[nind_e]=np.arange(len(nind_e))
    
    node_e=nodes[nind_e,:] #localized node set
    elem_e=nind_map_global_to_local[elements[mask_e,:]] #localized element set
    face_s=nind_map_global_to_local[faces[mask_s,:]] #localized face set
    
    nind_d=nind_map_global_to_local[mask_d] #localized node index
    nind_notd=nind_map_global_to_local[~mask_d] #localized node index
    
    #nflags_notd is a temporary variable to assist imposition of Dirichlet B.C
    #nonzero for nodes not on Dirichlet boundary nodes_e[nind_notd]
    nflags3_notd=np.zeros((len(node_e),3),dtype=int)
    nflags3_notd[nind_notd,:]=1
    
    mask3_d=(1-nflags3_notd).astype(bool)
    mask3_d=mask3_d.flatten(order='C')
    
    cxe33=cx33[mask_e,:,:] #element based
    cye33=cy33[mask_e,:,:] #element based
    cze33=cz33[mask_e,:,:] #element based
    
    alphaxe33=alphax33[mask_e,:,:] #element based
    alphaye33=alphay33[mask_e,:,:] #element based
    alphaze33=alphaz33[mask_e,:,:] #element based
    
    #anod33=a33[nind_e,:,:] #node based
    #fnod3=f3[nind_e,:] #node based
    ae33=a33[mask_e,:,:] #elemenet based
    fe3=f3[mask_e,:] #element based
    
    gnod3=g3[nind_e,:] #node based
    qnod33=q33[nind_e,:,:] #node based
    snod3=s3[nind_e,:] #node based
    
    nnode=len(node_e)
    nelem=len(elem_e)
    nd=len(nind_d)
    nnotd=len(nind_notd)
    ns=len(face_s)

    nt=(nelem*16+ns*9)*9 #three components/unknowns per node
    I33=np.zeros(nt) #holds the row indices
    J33=np.zeros(nt) #holds the col indices
    X33=np.zeros(nt,dtype=complex) #holds the element values
    X33_notd=np.zeros(nt) #nonzero for nodes not on Dirichlet boundary
    nt=0
    b3=np.zeros(nnode*3,dtype=complex)
    b3_notd=np.zeros(nnotd*3,dtype=complex)
    
    print('Assembling the system of equations')
    start=time.time()
    for i in range(nelem):
        sctr=elem_e[i,:4]
        xn=node_e[sctr,0]
        yn=node_e[sctr,1]
        zn=node_e[sctr,2]
        #Ke33,be3=build_Ke3(xn,yn,zn,cxe33[i,:,:],cye33[i,:,:],cze33[i,:,:],anod33[sctr,:,:],fnod3[sctr,:])
        Ke33,be3=build_Ke3(xn,yn,zn,cxe33[i,:,:],cye33[i,:,:],cze33[i,:,:],ae33[i,:,:],fe3[i,:],
                          alphaxe33[i,:,:],alphaye33[i,:,:],alphaze33[i,:,:])
        Ke3_notd=np.reshape(nflags3_notd[sctr,:].flatten(order='C'),(4*3,1))
        nt=nt+16*9
        
        #sctr holds the global node indices of the four nodes, size of (4,)
        #e.g. sctr=[0,1,2,3]
        #sctr3 holds the global unkonwn indicies on the four volume nodes, size of (4,3) or (12,)
        #three components/unknowns per node times four nodes
        #e.g. sctr3=[[0,1,2],[3,4,5],[6,7,8],[9,11,12]] or flattened
        sctr3=np.zeros(12,dtype=int) #three components/unknowns per node times four nodes
        for j in range(4):
            sctr3[j*3:(j+1)*3]=sctr[j]*3+np.arange(3)
        
        #I holds the global row indices of Ke_ij, e.g. [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]
        #J holds the global col indices of Ke_ij, e.g. [[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]]
        #X holds the entries of Ke_ij
        I33[nt-16*9:nt]=np.matlib.repmat(sctr3,4*3,1).flatten(order='F') #row indcies grow slower
        J33[nt-16*9:nt]=np.matlib.repmat(sctr3,4*3,1).flatten(order='C') #col indices grow faster
        X33[nt-16*9:nt]=Ke33.flatten(order='C') #stored in row major and Ke33_ij!=Ke33_ji
        X33_notd[nt-16*9:nt]=Ke3_notd.dot(Ke3_notd.T).flatten(order='C') #stored in row major
        b3[sctr3]=b3[sctr3]+be3
    
    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')
    
    print('Incoorprating the boundary condition of the third kind') #continue from here
    start=time.time()
    for i in range(ns):
        sctr=face_s[i,:3]        
        xn=nodes[sctr,0]
        yn=nodes[sctr,1]
        zn=nodes[sctr,2]
        Ks33,bs3=build_Ks3(xn,yn,zn,gnod3[sctr,:],qnod33[sctr,:,:])
        Ks3_notd=np.reshape(nflags3_notd[sctr,:].flatten(order='C'),(3*3,1))
        nt=nt+9*9
        
        #sctr holds the global node indices of the four nodes, size of (3,)
        #e.g. sctr=[0,1,2]
        #sctr3 holds the global unkonwn indicies on the three surface nodes, size of (3,3) or (9,)
        #three components/unknowns per node times three nodes
        #e.g. sctr3=[[0,1,2],[3,4,5],[6,7,8]] or flattened
        sctr3=np.zeros(9,dtype=int) #three components/unknowns per node times three nodes
        for j in range(3):
            sctr3[j*3:(j+1)*3]=sctr[j]*3+np.arange(3)
        
        #I holds the global row indices of Ks_ij, e.g. [[1,1,1],[2,2,2],[3,3,3]]
        #J holds the global col indices of Ks_ij, e.g. [[1,2,3],[1,2,3],[1,2,3]]
        #X holds the entries of Ks_ij
        I33[nt-9*9:nt]=np.matlib.repmat(sctr3,3*3,1).flatten(order='F') #row indices grow slower
        J33[nt-9*9:nt]=np.matlib.repmat(sctr3,3*3,1).flatten(order='C') #col indices grow faster
        X33[nt-9*9:nt]=Ks33.flatten(order='C') #stored in row major and Ks3_ij!=Ks3_ji
        X33_notd[nt-9*9:nt]=Ks3_notd.dot(Ks3_notd.T).flatten(order='C') #stored in row major
        b3[sctr3]=b3[sctr3]+bs3
    
    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')

    #build matrix K using I,J,X
    K33=csr_matrix((X33,(I33,J33)),shape=(nnode*3,nnode*3))
    K33_notd=csr_matrix((X33_notd,(I33,J33)),shape=(nnode*3,nnode*3))
    K33_notd=csr_matrix.sign(K33_notd)
    
    start=time.time()
    print('Incoorprating the Dirichlet boundary condition')
    sd3=snod3[nind_d,:].flatten(order='C')
    b3[mask3_d]=sd3

    for i in range(nnotd):
        sctr=nind_notd[i]
        #b[sctr]=b[sctr]-K[sctr,mask_d].dot(s)
        for j in range(3):
            b3_notd[i*3+j]=K33[sctr*3+j,mask3_d].dot(sd3)
            b3[sctr*3+j]=b3[sctr*3+j]-b3_notd[i*3+j]
        
    K33=K33.multiply(K33_notd) #nonzero entries i.e. K3.getnnz() get reduced
    nt=K33.getnnz()+nd*3
    I33=np.zeros(nt) #holds the row indices
    J33=np.zeros(nt) #holds the col indices
    X33=np.zeros(nt,dtype=complex) #holds the element values

    I33[:K33.getnnz()]=K33.nonzero()[0]
    J33[:K33.getnnz()]=K33.nonzero()[1]
    X33[:K33.getnnz()]=K33.data
    
    sctr3=np.zeros(nd*3)
    sctr3[0::3]=nind_d*3
    sctr3[1::3]=nind_d*3+1
    sctr3[2::3]=nind_d*3+2
    
    I33[K33.getnnz():]=sctr3
    J33[K33.getnnz():]=sctr3
    X33[K33.getnnz():]=np.ones(nd*3)
    K33=csr_matrix((X33,(I33,J33)),shape=(nnode*3,nnode*3))

    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')
    return K33,b3,b3_notd

#compute Ke and be for vector field (not tested so far)
def build_Ke3(xn,yn,zn,cx33,cy33,cz33,a33,f3,alphax33,alphay33,alphaz33):
    Ke33=np.zeros((4*3,4*3),dtype=complex)
    be3=np.zeros(4*3,dtype=complex)
    #row3=np.zeros((4*3,4*3))
    #col3=np.zeros((4*3,4*3))
    Je=np.zeros((4,4)) #basis function does not change
    
    Je[0,:]=1
    Je[1,:]=xn #x-coordinates of four nodes in an element
    Je[2,:]=yn
    Je[3,:]=zn
    invJe=np.linalg.inv(Je)
    detJe=np.linalg.det(Je)
    vol=detJe/6
    
    #see Eq 5.20 and 5.21 (Jin 2002 P171) for Ke and be
    #Ke should be modified for nonzero betax/betay/betaz
    #Ke[i,j]=(cx*invJe[i,1]*invJe[j,1]+cy*invJe[i,2]*invJe[j,2]+
    #         cz*invJe[i,3]*invJe[j,3]+a*(1+delta_ij)/20)*vol
    #be[i]=f*vol/4
    for i in range(12):
        i1=int(i/3) #i1^th node, i1=1,2,3
        k1=int(i)%3 #k1^th uknown, j1=1,2,3
        for j in range(12):
            j1=int(j/3)
            l1=int(j)%3
            #row3[i,j]=sctr[i1]*3+k1
            #col3[i,j]=sctr[j1]*3+l1
            
            tmp=1-np.abs(np.sign(i1-j1)) #delta(i1,j1)
            #Ke33[i,j]=(cx33[k1,l1]*invJe[i1,1]*invJe[j1,1]+cy33[k1,l1]*invJe[i1,2]*invJe[j1,2]+
            #          cz33[k1,l1]*invJe[i1,3]*invJe[j1,3]+a33[j1,k1,l1]*(1+tmp)/20)*vol
            Ke33[i,j]=(cx33[k1,l1]*invJe[i1,1]*invJe[j1,1]+cy33[k1,l1]*invJe[i1,2]*invJe[j1,2]+
                       cz33[k1,l1]*invJe[i1,3]*invJe[j1,3]+a33[k1,l1]*(1+tmp)/20)*vol
            Ke33[i,j]=Ke33[i,j]+(alphax33[k1,l1]*invJe[i1,1]+alphay33[k1,l1]*invJe[i1,2]+
                                 alphaz33[k1,l1]*invJe[i1,3])*vol/4
        #for j1 in range(4):
        #    tmp=1-np.abs(np.sign(i1-j1)) #delta(i1,j1)
        #    be3[i]=be3[i]+f3[j1,k1]*vol*(1+tmp)/20
        be3[i]=f3[k1]*vol/4
            
    return Ke33,be3

#compute Ks and bs for vector field (not tested so far)
def build_Ks3(xs,ys,zs,g3,q33):
    #modify this to accomodate matrix/vector PDE coefficients
    Ks3=np.zeros((3*3,3*3),dtype=complex)
    bs3=np.zeros(3*3,dtype=complex)
    #row3=np.zeros((3*3,3*3))
    #col3=np.zeros((3*3,3*3))
    
    a=[xs[1]-xs[0],ys[1]-ys[0],zs[1]-zs[0]]
    b=[xs[2]-xs[1],ys[2]-ys[1],zs[2]-zs[1]]
    c=np.cross(a,b)
    area=np.linalg.norm(c)/2
    
    #see Eq 5.32 and 5.33 (Jin 2002 P172) for Ks and bs
    #Ks[i,j]=q*(1+delta_ij)*area/12
    #bs[i]=g*area/3
    for i in range(9):
        i1=int(i/3) #i1^th node, i1=1,2,3
        k1=int(i)%3 #k1^th uknown, j1=1,2,3
        for j in range(9):
            j1=int(j/3)
            l1=int(j)%3
            #row3[i,j]=sctr[i1]*3+k1
            #col3[i,j]=sctr[j1]*3+l1
            
            tmp=1-np.abs(np.sign(i1-j1)) #delta(i1,j1)
            Ks3[i,j]=q33[j1,k1,l1]*(1+tmp)*area/12
        for j1 in range(3):
            tmp=1-np.abs(np.sign(i1-j1)) #delta(i1,j1)
            bs3[i]=bs3[i]+g3[j1,k1]*(1+tmp)*area/12
        #bs3[i]=g3*area/3 #to be modified
        
    return Ks3,bs3

#build system of equations for scalar field
def build_system(nodes,elements,faces,cx,cy,cz,a,f,g,q,s):
    nind_e=np.unique(elements[mask_e].flatten(order='C'))
    nind_map_global_to_local=-np.ones(len(nodes),dtype=int)
    nind_map_global_to_local[nind_e]=np.arange(len(nind_e))
    
    node_e=nodes[nind_e,:]
    elem_e=nind_map_global_to_local[elements[mask_e,:]]
    face_s=nind_map_global_to_local[faces[mask_s,:]]
    
    nind_d=nind_map_global_to_local[np.where(mask_d)[0]]
    nind_notd=nind_map_global_to_local[np.where(~mask_d)[0]]
    
    #nflags_notd is a temporary variable to assist imposition of Dirichlet B.C
    #nonzero for nodes not on Dirichlet boundary nodes_e[nind_notd]
    nflags_notd=np.zeros(len(node_e),dtype=int)
    nflags_notd[nind_notd]=1
    
    cxe=cx[mask_e]
    cye=cy[mask_e]
    cze=cz[mask_e]
    
    #ae=a[mask_e]
    #fe=f[mask_e]
    anod=a[nind_e]
    fnod=f[nind_e]
    
    gnod=g[nind_e]
    qnod=q[nind_e]
    sd=s[mask_d]

    nnode=len(node_e)
    nelem=len(elem_e)
    nd=len(nind_d)
    nnotd=len(nind_notd)
    ns=len(face_s)
    
    #nt=nelem*16+ns*9+nnotd*nd*2
    nt=nelem*16+ns*9
    I=np.zeros(nt) #holds the row indices
    J=np.zeros(nt) #holds the col indices
    X=np.zeros(nt) #holds the element values
    X_notd=np.zeros(nt) #nonzero for nodes not on Dirichlet boundary
    nt=0
    b=np.zeros(nnode)
    b_notd=np.zeros(nnotd)
    
    print('Assembling the system of equations')
    start=time.time()
    for i in range(nelem):
        sctr=elem_e[i,:4]
        xn=node_e[sctr,0]
        yn=node_e[sctr,1]
        zn=node_e[sctr,2]
        Ke,be=build_Ke(xn,yn,zn,cxe[i],cye[i],cze[i],anod[sctr],fnod[sctr])
        Ke_notd=np.reshape(nflags_notd[sctr],(4,1))
        nt=nt+16
        
        #I holds the global row indices of Ke_ij, e.g. [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]
        #J holds the global col indices of Ke_ij, e.g. [[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]]
        #X holds the entries of Ke_ij
        I[nt-16:nt]=np.matlib.repmat(sctr,4,1).flatten(order='F')
        J[nt-16:nt]=np.matlib.repmat(sctr,4,1).flatten(order='C')
        X[nt-16:nt]=Ke.flatten(order='C') #This oder is not important because Ke_ij=Ke_ji
        X_notd[nt-16:nt]=Ke_notd.dot(Ke_notd.T).flatten(order='C') #This order does not matter neither
        b[sctr]=b[sctr]+be
    
    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')
    
    print('Incoorprating the boundary condition of the third kind')
    start=time.time()
    for i in range(ns):
        sctr=face_s[i,:3]
        xn=node_e[sctr,0]
        yn=node_e[sctr,1]
        zn=node_e[sctr,2]
        Ks,bs=build_Ks(xn,yn,zn,gnod[sctr],qnod[sctr])
        Ks_notd=np.reshape(nflags_notd[sctr],(3,1))
        nt=nt+9
        
        #I holds the global row indices of Ks_ij, e.g. [[1,1,1],[2,2,2],[3,3,3]]
        #J holds the global col indices of Ks_ij, e.g. [[1,2,3],[1,2,3],[1,2,3]]
        #X holds the entries of Ks_ij
        I[nt-9:nt]=np.matlib.repmat(sctr,3,1).flatten(order='F')
        J[nt-9:nt]=np.matlib.repmat(sctr,3,1).flatten(order='C')
        X[nt-9:nt]=Ks.flatten(order='C') #This order is not important because Ks_ij=Ks_ji
        X_notd[nt-9:nt]=Ks_notd.dot(Ks_notd.T).flatten(order='C') #This order does not matter neither
        b[sctr]=b[sctr]+bs
    
    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')

    #build matrix K using I,J,X
    #remember to manipulate elements in K using boolean index
    K=csr_matrix((X,(I,J)),shape=(nnode,nnode))
    K_notd=csr_matrix((X_notd,(I,J)),shape=(nnode,nnode))
    K_notd=csr_matrix.sign(K_notd)
    
    start=time.time()
    print('Incoorprating the Dirichlet boundary condition')
    b[nind_d]=sd

    for i in range(nnotd):
        sctr=nind_notd[i]
        #b[sctr]=b[sctr]-K[sctr,nind_d].dot(sd)
        b_notd[i]=K[sctr,nind_d].dot(sd)
        b[sctr]=b[sctr]-b_notd[i]
        
    K=K.multiply(K_notd) #nonzero entries i.e. K.getnnz() get reduced
    nt=K.getnnz()+nd
    I=np.zeros(nt) #holds the row indices
    J=np.zeros(nt) #holds the col indices
    X=np.zeros(nt) #holds the element values

    I[:K.getnnz()]=K.nonzero()[0]
    J[:K.getnnz()]=K.nonzero()[1]
    X[:K.getnnz()]=K.data

    I[K.getnnz():]=nind_d
    J[K.getnnz():]=nind_d
    X[K.getnnz():]=np.ones(nd)
    K=csr_matrix((X,(I,J)),shape=(nnode,nnode))

    elapsed=time.time()-start
    print('Time elapsed ',elapsed,'sec')
    print('')
    return K,b,b_notd

#compute Ke and be for scalar field
def build_Ke(xn,yn,zn,cx,cy,cz,a,f):
    Ke=np.zeros((4,4))
    be=np.zeros(4)
    Je=np.zeros((4,4))
    
    Je[0,:]=1
    Je[1,:]=xn #x-coordinates of four nodes in an element
    Je[2,:]=yn
    Je[3,:]=zn
    invJe=np.linalg.inv(Je)
    detJe=np.linalg.det(Je)
    vol=detJe/6
    
    #see Eq 5.20 and 5.21 (Jin 2002 P171) for Ke and be
    #Ke should be modified in case of nonzero betax/betay/betaz
    for i in range(4):
        for j in range(4):
            tmp=1-np.abs(np.sign(i-j))
            Ke[i,j]=(cx*invJe[i,1]*invJe[j,1]+cy*invJe[i,2]*invJe[j,2]+
                     cz*invJe[i,3]*invJe[j,3]+a[j]*(1+tmp)/20)*vol
            be[i]=be[i]+f[j]*vol*(1+tmp)/20
    
    return Ke,be

#compute Ks and bs for scalar field
def build_Ks(xs,ys,zs,g,q):
    Ks=np.zeros((3,3))
    bs=np.zeros(3)
    
    a=[xs[1]-xs[0],ys[1]-ys[0],zs[1]-zs[0]]
    b=[xs[2]-xs[1],ys[2]-ys[1],zs[2]-zs[1]]
    c=np.cross(a,b)
    area=np.linalg.norm(c)/2
    
    #see Eq 5.32 and 5.33 (Jin 2002 P172) for Ks and bs
    for i in range(3):
        for j in range(3):
            tmp=1-np.abs(np.sign(i-j))
            Ks[i,:]=q[j]*area*(1+tmp)/12
            bs[i]=bs[i]+g[j]*area*(1+tmp)/12
        #bs[i]=g*area/3 #may need to be modified
        
    return Ks,bs

print('Done')

# Compute itermediate variables

In [ ]:
def an_sol(x,a,rel_perm,tK,cinf,zval,zeta,gradflag=0):
    echarge=1.602e-19 #elementary charge [C]
    perm0=8.85e-12 #vacuum permittivity [F/m]
    kB=1.381e-23 #Boltzmann's constant [J/K]
    kA=6.022e23 #Avogadro constant [1/mol]

    debye_len=np.sqrt(perm0*rel_perm*kB*tK/2/(zval*echarge)**2/kA/cinf)
    print('DEBYE LENGTH IS:',debye_len*1e9,'nm')
    
    if gradflag:
        zeta=zeta/(1/debye_len+1/a)/perm0/rel_perm
    
    u=zeta*a*np.exp((a-x)/debye_len)/x
    du=-u*(1/debye_len+1/x) #du/dr
    return u,du

u3=np.load('sphere_s41_u3_4.npy')


# Setup PDE coefficients

In [ ]:
import numpy as np

#modified on 6/23/2021
#setup a in main.py with shape changed from (nelem,) to (nnode,)
#setup a in main.py with shape changed from (nelem,) to (nnode,)
#replaced perm with rel_perm
#changed cinf unit from m^-3 to mol*m^-3 by removing kA from its value

#modified on 8/18/2021
#changed g and q from element based to node based

mesh_prefix='sphere'
outfile='sphere_s41.npz'

#set physical constants and properties
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kB=1.381e-23 #Boltzmann's constant [J/K]
kA=6.022e23 #Avogadro constant [1/mol]

rel_perm=80 #relative permittivity of bulk electrolyte [F/m]
tK=293 #room/ambient temperature [K]
cinf=1 #ion concentration in bulk electrolyte [mol/m^3]
zval=1 #ion valence in bulk electrolyte
# cinf_pos=0.01 #ion concentration of positive charges
# cinf_neg=0.01 #concentration of negative charges
# zval_pos=1 #positive ion valence
# zval_neg=1 #negative ion valence

rel_perm=78.5
tK=298
cinf=100
zval=1

#load mesh
print('Reading %s.1.node'%mesh_prefix)
nodes=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=(1,2,3))
node_flags=np.genfromtxt(mesh_prefix+'.1.node',skip_header=1,skip_footer=1,usecols=5,dtype='int')

print('Reading %s.1.ele'%mesh_prefix)
elements=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=(1,2,3,4),dtype='int')
zones=np.genfromtxt(mesh_prefix+'.1.ele',skip_header=1,usecols=5,dtype='int')

print('Reading %s.1.face'%mesh_prefix)
faces=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=(1,2,3),dtype='int')
face_flags=np.genfromtxt(mesh_prefix+'.1.face',skip_header=1,usecols=4,dtype='int')

#adjust indices to start from zero
elements=elements-1
faces=faces-1

# #translate z coordinate
# print('Translating Z coordinate')
# nodes[:,2]=-(np.power(10,-nodes[:,2]/1000*3)-1) #problem dependent
dist1=np.sqrt(nodes[:,0]**2+nodes[:,1]**2+nodes[:,2]**2)
mask=dist1>5.0
dist2=np.zeros_like(dist1)
dist2[mask]=10**((dist1[mask]-5)/50*2)+5-1

print(min(dist1[mask]),max(dist1[mask]))
print(min(dist2[mask]),max(dist2[mask]))
nodes[mask,0]=nodes[mask,0]/dist1[mask]*dist2[mask]
nodes[mask,1]=nodes[mask,1]/dist1[mask]*dist2[mask]
nodes[mask,2]=nodes[mask,2]/dist1[mask]*dist2[mask]

#scale nodes from meter to nano-meter
nodes=nodes*1e-9

nnode=len(nodes)
nelem=len(elements)
nface=len(faces)
print('THE NUMBER OF NODES IS: %d'%nnode)
print('THE NUMBER OF ELEMENTS IS: %d'%nelem)
print('THE NUMBER OF FACES IS: %d'%nface)

#set boolean indicies for volume elements, S1 nodes, and S2 elements
mask_e=zones<=4 #all elements
# mask_d=node_flags==1 #boolean node indices True for Dirichlet nodes
# mask_s=face_flags==1 #boolean face indices True for third kind surface elements
mask_d=(node_flags==1)|(node_flags==2) #outer boundary may need to be updated
#mask_s=np.zeros_like(face_flags) #empty for this case
mask_s=face_flags==20 #interface between zone 1 and 2

#initialize scalar PDE coefficients
print('Setting PDE coefficients')
cx=perm0*rel_perm*np.ones(nelem)
cy=perm0*rel_perm*np.ones(nelem)
cz=perm0*rel_perm*np.ones(nelem)

cx[(zones==1)|(zones==2)]=perm0*4.5
cy[(zones==1)|(zones==2)]=perm0*4.5
cz[(zones==1)|(zones==2)]=perm0*4.5

alphax=np.zeros(nelem) #unused so far
alphay=np.zeros(nelem) #unused so far
alphaz=np.zeros(nelem) #unused so far

betax=np.zeros(nelem) #unused so far
betay=np.zeros(nelem) #unused so far
betaz=np.zeros(nelem) #unused so far

gammax=np.zeros(nelem) #unused so far
gammay=np.zeros(nelem) #unused so far
gammaz=np.zeros(nelem) #unused so far

a=np.zeros(nnode)
f=np.zeros(nnode)
# a='pb' #will be setup in main.py with shape (nnode,)
# f='pb' #will be setup in main.py with shape (nnode,)

g=np.zeros(nnode) #length of ns for essential entries
q=np.zeros(nnode) #length of ns for essential entries
s=np.zeros(nnode) #length of nd for essential entries

#set Dirichlet B.C on solid interface
#for monovalent ions (zval=1), we can use linear PB equation for phi_e<<26mV
#for divalent ions (zval=2), we can use linear PB equation for phi_e<<13 mv
#s[node_flags==10]=0.1 #phi_e=0.1V
nind_s=np.unique(faces[face_flags==20].flatten(order='C'))
g[nind_s]=0.01 #surface charge density [C/m^2]

#initialize vector PDE coefficients
cx33=np.zeros((nelem,3,3)) #PDE term coefficient
cy33=np.zeros((nelem,3,3)) #PDE term coefficient
cz33=np.zeros((nelem,3,3)) #PDE term coefficient

cx33[zones>=3,0,0]=perm0*rel_perm
cx33[zones>=3,1,1]=perm0*rel_perm
cx33[zones>=3,2,2]=perm0*rel_perm

cx33[zones<=2,0,0]=perm0*4.5
cx33[zones<=2,1,1]=perm0*4.5
cx33[zones<=2,2,2]=perm0*4.5

cy33=np.copy(cx33)
cz33=np.copy(cx33)

alphax33=np.zeros((nelem,3,3)) #PDE term coefficient
alphay33=np.zeros((nelem,3,3)) #PDE term coefficient
alphaz33=np.zeros((nelem,3,3)) #PDE term coefficient

betax33=np.zeros((nelem,3,3)) #PDE term coefficient
betay33=np.zeros((nelem,3,3)) #PDE term coefficient
betaz33=np.zeros((nelem,3,3)) #PDE term coefficient

gammax33=np.zeros((nelem,3,3)) #PDE term coefficient
gammay33=np.zeros((nelem,3,3)) #PDE term coefficient
gammaz33=np.zeros((nelem,3,3)) #PDE term coefficient

a33=np.zeros((nelem,3,3),dtype=complex) #PDE term coefficient
f3=np.zeros((nelem,3),dtype=complex) #source term coefficient
g3=np.zeros((nnode,3)) #Neumann boundary condition coefficient
q33=np.zeros((nnode,3,3)) #Neumann boundary condition coefficient
s3=np.zeros((nnode,3)) #Dirichlet boundary condition coefficient

nind_s=np.unique(faces[face_flags==20].flatten(order='C'))
g3[nind_s,0]=0.01 #surface charge density [C/m^2]
g3[nind_s,1]=0.01 #surface charge density [C/m^2]
g3[nind_s,2]=0.01 #surface charge density [C/m^2]

#save PDE coefficients to a file
print('Saving PDE coefficients to %s'%outfile)
np.savez(outfile,nodes=nodes,elements=elements,faces=faces,
         node_flags=node_flags,zones=zones,face_flags=face_flags,
         mask_e=mask_e,mask_d=mask_d,mask_s=mask_s,
         cx=cx,cy=cy,cz=cz,
         alphax=alphax,alphay=alphay,alphaz=alphaz,
         betax=betax,betay=betay,betaz=betaz,
         gammax=gammax,gammay=gammay,gammaz=gammaz,
         a=a,f=f,g=g,q=q,s=s,
         cx33=cx33,cy33=cy33,cz33=cz33,
         alphax33=alphax33,alphay33=alphay33,alphaz33=alphaz33,
         betax33=betax33,betay33=betay33,betaz33=betaz33,
         gammax33=gammax33,gammay33=gammay33,gammaz33=gammaz33,
         a33=a33,f3=f3,g3=g3,q33=q33,s3=s3,
         rel_perm=rel_perm,tK=tK,cinf=cinf,zval=zval)
print('')
